# 01: Search Fundamentals

## Learning Objectives
- Understand the components of a search problem
- Learn about state spaces and search trees
- Implement a basic search problem

## 1. Introduction to Search Problems

Search is fundamental to AI. A search problem consists of:
- **Initial state**: Where we start
- **Actions**: What we can do in each state
- **Transition model**: How actions change states
- **Goal test**: How we know we've succeeded
- **Path cost**: Cost of a sequence of actions

In [ ]:
# Import required modules
import sys
sys.path.append('../src')
from typing import Any, List, Tuple

## 2. The SearchProblem Interface

In [ ]:
class SearchProblem:
    """Abstract base class for search problems"""
    
    def get_start_state(self) -> Any:
        """Returns the initial state for the search problem"""
        raise NotImplementedError
    
    def is_goal_state(self, state: Any) -> bool:
        """Returns True if the state is a goal state"""
        raise NotImplementedError
    
    def get_successors(self, state: Any) -> List[Tuple[Any, str, float]]:
        """Returns list of (successor, action, cost) tuples"""
        raise NotImplementedError
    
    def get_cost_of_actions(self, actions: List[str]) -> float:
        """Returns the total cost of a sequence of actions"""
        raise NotImplementedError

## 3. Example: Grid Navigation Problem

In [ ]:
class GridSearchProblem(SearchProblem):
    """Grid navigation with obstacles"""
    
    def __init__(self, grid: List[List], start: Tuple[int, int], goal: Tuple[int, int]):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.rows = len(grid)
        self.cols = len(grid[0]) if grid else 0
    
    def get_start_state(self):
        return self.start
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def get_successors(self, state):
        successors = []
        row, col = state
        
        moves = [
            ((-1, 0), 'UP', 1.0),
            ((1, 0), 'DOWN', 1.0),
            ((0, -1), 'LEFT', 1.0),
            ((0, 1), 'RIGHT', 1.0)
        ]
        
        for (dr, dc), action, cost in moves:
            new_row, new_col = row + dr, col + dc
            
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.grid[new_row][new_col] != 1):
                
                successors.append(((new_row, new_col), action, cost))
        
        return successors
    
    def get_cost_of_actions(self, actions):
        return len(actions)

## 4. Create and Test a Sample Problem

In [ ]:
# Create a sample grid world (0=free, 1=obstacle)
sample_grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_pos = (0, 0)
goal_pos = (4, 4)

problem = GridSearchProblem(sample_grid, start_pos, goal_pos)

# Test the problem
print(f"Start state: {problem.get_start_state()}")
print(f"Is goal? {problem.is_goal_state(start_pos)}")
print("\nSuccessors from start:")
for successor, action, cost in problem.get_successors(start_pos):
    print(f"  {action}: move to {successor} (cost={cost})")

## 5. Search Node Class

In [ ]:
class SearchNode:
    """Node in the search tree"""
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1
    
    def get_path(self):
        path = []
        node = self
        while node:
            path.append(node.state)
            node = node.parent
        return list(reversed(path))
    
    def get_actions(self):
        actions = []
        node = self
        while node.parent:
            actions.append(node.action)
            node = node.parent
        return list(reversed(actions))

## 6. Exercise 1: 8-Puzzle Problem (COMPLETED)

In [ ]:
class EightPuzzleProblem(SearchProblem):
    """8-puzzle: 3x3 grid with tiles 1-8 and one blank (0)"""
    
    def __init__(self, initial_state, goal_state=None):
        self.initial = initial_state
        self.goal = goal_state or (1, 2, 3, 4, 5, 6, 7, 8, 0)
    
    def get_start_state(self):
        return self.initial
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def get_successors(self, state):
        successors = []
        blank_pos = state.index(0)
        row, col = blank_pos // 3, blank_pos % 3
        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]
        for dr, dc, action in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                new_state = list(state)
                new_state[blank_pos], new_state[new_pos] = new_state[new_pos], new_state[blank_pos]
                successors.append((tuple(new_state), action, 1))
        return successors
    
    def get_cost_of_actions(self, actions):
        return len(actions)

# Test the 8-puzzle
puzzle = EightPuzzleProblem((1, 2, 3, 4, 0, 6, 7, 5, 8))
print("8-Puzzle successors:")
for successor, action, cost in puzzle.get_successors(puzzle.get_start_state()):
    print(f"  {action}: {successor}")

## 7. Exercise 2: Variable Cost Grid Problem (COMPLETED)

In [ ]:
class VariableCostGridProblem(GridSearchProblem):
    """Grid navigation with diagonal moves (cost √2)"""
    
    def get_successors(self, state):
        successors = []
        row, col = state
        
        # Orthogonal + diagonal moves
        moves = [
            ((-1, 0), 'UP', 1.0),
            ((1, 0), 'DOWN', 1.0),
            ((0, -1), 'LEFT', 1.0),
            ((0, 1), 'RIGHT', 1.0),
            ((-1, -1), 'UP-LEFT', 1.414),
            ((-1, 1), 'UP-RIGHT', 1.414),
            ((1, -1), 'DOWN-LEFT', 1.414),
            ((1, 1), 'DOWN-RIGHT', 1.414)
        ]
        
        for (dr, dc), action, cost in moves:
            new_row, new_col = row + dr, col + dc
            
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.grid[new_row][new_col] != 1):
                
                successors.append(((new_row, new_col), action, cost))
        
        return successors

# Test variable cost grid
vcg_problem = VariableCostGridProblem(sample_grid, (0, 0), (4, 4))
print("\nVariable cost successors from (2,2):")
for successor, action, cost in vcg_problem.get_successors((2, 2)):
    print(f"  {action}: {successor} (cost={cost})")

## 8. Key Takeaways

1. **Search problems** are defined by states, actions, and goals
2. **State space** can be much larger than what we can explore
3. **Search trees** represent paths through the state space
4. **Different problems** require different state representations
5. **Cost functions** determine optimal solutions

Both TODO implementations are now complete:
- **EightPuzzle**: Sliding tile mechanics with blank position tracking
- **VariableCostGrid**: Diagonal movement with proper √2 costs